In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"
cohort = "GSE66795"

# Input paths
in_trait_dir = "../../input/GEO/Sjögrens_Syndrome"
in_cohort_dir = "../../input/GEO/Sjögrens_Syndrome/GSE66795"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/GSE66795.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE66795.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE66795.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information and sample characteristics, this dataset likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (pSS) can be inferred from "patient group" (key 2)
trait_row = 2  # 'patient group: Control', 'patient group: Patient'

# Age is not available in the sample characteristics
age_row = None  

# Gender is available (key 3), but there's only one value ('gender: Female')
# Since all subjects are female, this is a constant feature and not useful for association analysis
gender_row = None  

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert patient group to binary trait (Sjögren's Syndrome)"""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        if value.lower() == "patient":
            return 1  # Has Sjögren's Syndrome
        elif value.lower() == "control":
            return 0  # Doesn't have Sjögren's Syndrome
    return None

def convert_age(value):
    """Convert age to continuous value"""
    # Age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)"""
    # Gender is constant (all female) so not useful for analysis
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info with is_final=False for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (since trait_row is not None)
if trait_row is not None:
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the selected clinical features as a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Analysis of gene identifiers
# The gene identifiers shown (ILMN_xxxxxxx) are Illumina probe IDs, not human gene symbols.
# These are microarray probe identifiers from Illumina BeadArray platforms
# These need to be mapped to standard gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the relevant columns for gene mapping
# From the preview, we can see:
# - 'ID' column contains Illumina probe IDs (ILMN_xxxxxx) that match gene_data indices
# - 'Symbol' column contains the gene symbols we need

# 2. Extract the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Gene mapping preview:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("Gene expression data preview (first 5 genes, first 5 samples):")
print(gene_data.iloc[:5, :5])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df.T, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Rename the unnamed index 0 to the trait name
linked_data = linked_data.rename(index={0: trait})
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from whole blood of pSS patients and healthy controls."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# 3. Transform clinical data to have trait as column
sample_ids = clinical_df.columns
trait_values = clinical_df.iloc[0].values
clinical_df_restructured = pd.DataFrame({trait: trait_values}, index=sample_ids)
print(f"Restructured clinical data shape: {clinical_df_restructured.shape}")
print(clinical_df_restructured.head())

# 4. Properly format gene expression data (transpose to have samples as rows)
gene_expr_for_linking = normalized_gene_data.T
print(f"Transposed gene data shape: {gene_expr_for_linking.shape}")

# 5. Merge clinical and gene expression data on sample IDs
linked_data = clinical_df_restructured.join(gene_expr_for_linking)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head(3))

# 6. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 7. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 8. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from whole blood samples of pSS patients and healthy controls."
)

# 9. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")